In [10]:
!pip install pyomo ortools

import numpy as np

from pyomo.environ import *
from pyomo.gdp import *

from ortools.constraint_solver import pywrapcp, routing_enums_pb2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
TASKS = {
    'clean_times': [
        [0, 11, 7, 13, 11],
        [5, 0, 13, 15, 15],
        [13, 15, 0, 23, 11],
        [9, 13, 5, 0, 3],
        [3, 7, 7, 7, 0]
    ],
    'num_machines': 1,
    'start': 0,
    'mix_times': [40, 35, 45, 32, 50]
}

Το πρόβλημα που μας παρουσιάζεται απαιτεί την εύρεση του ελάχιστου χρόνου που απαιτείται για την παραγωγή μερικών χρωμάτων. Η παραγωγή αυτή απαιτεί πρώτα τον καθαρισμό του δοχείου και έπειτα την μίξη του χρώματος. Η μίξη του χρώματος απαιτεί κατ'ελάχιστο περισσότερο χρόνο από τον κατά μέγιστο καθαρισμό. Για τον παραπάνω λόγο πρόβλημα μπορεί να απλοποιηθεί καθώς δεν πρόκειται να 'βολέψει' να ξεκινήσει η ανάμιξη ενός χρώματος όταν θα περιμένει κάποιο άλλο να μπεί στο μηχάνημα. Λόγω των παραπάνω, το πρόβλημα μπορεί να λυθεί με παρόμοιο τρόπο με το πρόβλημα του πλανόδιου πωλητή, ή ένα πρόβλημα διάσχισης γράφου. 

In [16]:
# Συνάρτηση που βρίσκει την 'απόσταση' δύο κελιών, που στην δική μας περίπτωση είναι ο χρόνος εναλλαγής δύο χρωμάτων
def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return TASKS['clean_times'][from_node][to_node]

In [20]:
def print_solution(manager, routing, solution):
    incompressible_time = np.sum(TASKS['mix_times']) #Χρόνος ανάμειξης που δεν μπορεί να βελιστοποιηθεί με κανένα τρόπο
    print(f'Total: {solution.ObjectiveValue() + incompressible_time} minutes')
    
    index = routing.Start(TASKS['start'])
    plan_output = f'Best flow: '
    cleaning_times = 0

    while not routing.IsEnd(index):
        plan_output += f'{manager.IndexToNode(index)+ 1} -> '
        previous_index = index
        index = solution.Value(routing.NextVar(index)) 
        cleaning_times += routing.GetArcCostForVehicle(previous_index, index, 0)

    plan_output += f'{manager.IndexToNode(index)+ 1}'

    print(plan_output)    

Ορίζουμε ένα έτοιμο μοντέλο της `pywrapcp` βιβλιοθήκης το οποίο είναι φτιαγμένο για να λύνει προβλήματα όπως το πρόβλημα του πλανόδιου πωλητή. Το μοντέλο χρησιμοποιεί τη στρατηγική `PATH_CHEAPEST_ARC` η οποία δημιουργεί μια αρχική διαδρομή για τον επιλύτη προσθέτοντας επανειλημμένα ακμές με το μικρότερο βάρος που δεν οδηγούν σε κόμβο που έχει επισκεφθεί προηγουμένως.

In [15]:
manager = pywrapcp.RoutingIndexManager(len(TASKS['clean_times']),
                                       TASKS['num_machines'], 
                                       TASKS['start'])
routing = pywrapcp.RoutingModel(manager)

transit_callback_index = routing.RegisterTransitCallback(distance_callback)

routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

search_parameters = pywrapcp.DefaultRoutingSearchParameters()

search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

solution = routing.SolveWithParameters(search_parameters)


In [21]:
print_solution(manager, routing, solution)

Total: 243 minutes
Best flow: 1 -> 4 -> 3 -> 5 -> 2 -> 1
